# Dissolution energy of a molecule immerse in a dielectric medium

## Background

In a general description, the energy of an immerse molecule can be calculated by considering two main regions: 
the molecule region, in which the atoms radius doesn't allow to access the water molecules and salt ions surrounding it. The exterior region is an infinite and continuous saltwater solvent.
The model proposses a PDE system, the first equation is a Poisson field with a set of atoms (molecule to study), and the second is a mod-Helmholtz/Poisson-Boltzmann/Yukawa field simulating a simplified model of saltwater surrounding the molecule. 
So we can calculate the electrostatic potential $\phi$ solving the following system.

**Interior**
$$ \nabla^2 \phi_1 (x) = \sum_k q_k \delta (x_k)  $$
**Exterior**
$$ \nabla^2 \phi_2 (x) - \kappa \phi_2 (x) = 0  $$

with $q_k$ and $x_k$ as charge and position of each molecule atom, $\kappa$ is the inverse of Debye length

** Boundary Condition **
$$ \phi_{1} = \phi_{2} 
\hspace{40pt}
\epsilon_{1} \frac{\partial \phi_{1}}{\partial \textrm{ n}} = \epsilon_{2} \frac{\partial \phi_{2}}{\partial \textrm{ n}} $$

In the integral formulation appear different Green's functions corresponding to each field.

**Interior**
$$ \frac{\phi(x)}{2} + 
\int_\Gamma \frac{\partial G_L}{\partial \textrm{ n}}(x,x_\Gamma) \phi (x_\Gamma) \textrm{d} x_\Gamma - 
\int_\Gamma G_L\left( x,x_\Gamma \right) \frac{\partial \phi}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma
= \sum_k \frac{q_k}{4\pi |x-x_k|} $$

**Exterior**
$$ \frac{\phi(x)}{2} - 
\int_\Gamma \frac{\partial G_Y}{\partial \textrm{ n}}(x,x_\Gamma) \phi (x_\Gamma) \textrm{d} x_\Gamma + 
\int_\Gamma G_Y(x,x_\Gamma) \frac{\partial \phi}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma 
= 0$$

Applying boundary condition in terms of $\phi_1$ and writing the system in vector form, we obtain the problem to solve

$$
\begin{bmatrix}
    \frac{I}{2} + K_L &  -V_L \\
    \frac{I}{2} - K_Y &  \frac{\epsilon_1}{\epsilon_2}V_Y
\end{bmatrix}
\left\{
\begin{matrix}
    \phi \\
    \frac{\partial \phi}{\partial \textrm{ n}}
\end{matrix}
\right\}
= \left\{
\begin{matrix}
    \sum_k q_k \\
    0
\end{matrix}
\right\}
$$

once obtained the result, the potential at the atoms positions can be obtained with a direct evaluation.

$$\phi_k = 
\int_\Gamma G_L(x_k,x_\Gamma) \frac{\partial \phi}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma - 
\int_\Gamma \frac{\partial G_L}{\partial \textrm{ n}}(x_k,x_\Gamma) \phi (x_\Gamma) \textrm{d} x_\Gamma $$
       
and finally, the dissolution energy can be obtained integrating over the molecule region
       
$$ E = \frac{1}{2} \int_\Omega q_k \phi(x_k) \textrm{ d }x_\Omega $$

For deeper understanding of the method and its derivation, please go to **here**[**?**]

## Implementation

In this example we use a *molecule* 5PTI, a structure of bovine pancreatic trypsin inhibitor.[**?**]

### Excluded surface

The surface used to delimit both regions, its called Solvent Excluded Surface, and can be obtain with programs like `msms` or `nanoshaper`[**?**] and formated with the `GridFactory()` method. For simplicity in this example the geometry its imported. 

In [1]:
import numpy as np
import bempp.api

grid = bempp.api.import_grid('5pti_d1.msh')
grid.plot()

### Electrostatic potential on surface
The charges and positions of the set of atoms, can be obtain from a `.pqr` file, obtained from files availables on the Protein Data Bank [**?**].
To calculate the RHS of the system its needed proyect every atom charge to every boundary element, using the `GridFunction` method.

In [2]:
q, x_q = np.array([]), np.empty((0,3))

ep_in = 4.
ep_ex = 80.
k = 0.125

# Read charges and coordinates from the .pqr file
molecule_file = open('5pti.pqr', 'r').read().split('\n')
for line in molecule_file:
    line = line.split()
    if len(line)==0 or line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack(( x_q, np.array(line[5:8]).astype(float) ))

# Function to calculate the potential by charges at boundary
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    result[:] = np.sum(q/np.linalg.norm( x - x_q, axis=1 ))/(4*np.pi*ep_in)

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)
charged_grid_fun.plot()

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])

### Operators and Matrix

The following lines creates the operator over the previously defined spaces, and assemble the equation system.

In [3]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We use `gmres` of `scipy` to solve the system, for a better convergence study of the method is used an iteration counter and a frame registration.

In [5]:
import inspect
from scipy.sparse.linalg import gmres
array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, 
                tol=1e-3, maxiter=500, restart = 1000)

print("The linear system was solved in {0} iterations".format(it_count))

It: 1 Error 9.99E-01
It: 2 Error 3.52E-01
It: 3 Error 3.31E-01
It: 4 Error 2.20E-01
It: 5 Error 2.05E-01
It: 6 Error 1.41E-01
It: 7 Error 1.22E-01
It: 8 Error 1.01E-01
It: 9 Error 9.12E-02
It: 10 Error 7.60E-02
It: 11 Error 6.97E-02
It: 12 Error 5.68E-02
It: 13 Error 5.02E-02
It: 14 Error 4.21E-02
It: 15 Error 3.88E-02
It: 16 Error 3.38E-02
It: 17 Error 3.03E-02
It: 18 Error 2.80E-02
It: 19 Error 2.45E-02
It: 20 Error 2.31E-02
It: 21 Error 2.02E-02
It: 22 Error 1.86E-02
It: 23 Error 1.68E-02
It: 24 Error 1.57E-02
It: 25 Error 1.45E-02
It: 26 Error 1.35E-02
It: 27 Error 1.29E-02
It: 28 Error 1.17E-02
It: 29 Error 1.12E-02
It: 30 Error 9.94E-03
It: 31 Error 9.72E-03
It: 32 Error 8.41E-03
It: 33 Error 8.19E-03
It: 34 Error 7.56E-03
It: 35 Error 7.29E-03
It: 36 Error 6.81E-03
It: 37 Error 6.41E-03
It: 38 Error 6.22E-03
It: 39 Error 5.82E-03
It: 40 Error 5.75E-03
It: 41 Error 5.25E-03
It: 42 Error 5.17E-03
It: 43 Error 4.75E-03
It: 44 Error 4.62E-03
It: 45 Error 4.33E-03
It: 46 Error 4.18E-

The two following `GridFunction` stores the calculated boundary potential data (separate $\phi$ and $\frac{\partial \phi}{\partial n}$) so it can be visualizated. 

In [7]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
solution_dirichl.plot()

/usr/lib/python2.7/dist-packages/numpy/core/numeric.py:694: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)


### Energy calculaton

Its possible obtain the potential in the atoms position using the `operators.potential` method 

$$\phi_k = K_L\left[ \frac{\partial \phi}{\partial n} \right] - V_L \left[ \phi \right] $$ 
       
and the total dissolution energy can be reduced from the integral to the sum of the atoms potentials and charges

$$E = \frac{1}{2} \sum_k \phi_k q_k$$

In [8]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 2*np.pi*332.064*np.sum(q*phi_q).real
print("Total dissolution Energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total dissolution Energy: -352.84 [kcal/Mol]
